# Spotify Lab Example

The library uses Pandas and a single DataFrame for your library. All your "liked" music.

In [ ]:
%load_ext autoreload
%autoreload 2

import requests
from IPython.display import Image
from spotify import Spotify
from pandas import DataFrame as df
import pandas as pd
from datetime import datetime,timedelta,date
import json
import logging
import os

auth = os.environ.get("SPOTIFY_TOKEN")

logging.basicConfig()
defaultLogger = logging.getLogger()
defaultLogger.setLevel(logging.ERROR)
logger = logging.getLogger('SpotifyLab')
logger.setLevel(logging.DEBUG)

# spot is our connection to Spotify API
spot = Spotify(auth,  logger = logger)
Image(url=spot.imageUrl, width=50)

## Fetch the Library
Initialize the Spotify object with a `logger` and fetch the library DataFrame. We're using three playlists for manual lists:

* **Noise** - tracks to ignore when creating auto playlists based on how recently the track was added
* **Holiday Songs** - Christmas (or other) holiday music that is only played at holiday time
* **December Songs** - Other tracks to use with your holiday playlists but not exclusively holiday music

In [ ]:
# Read the library into a Data Frame
lib = spot.fetchLibraryDataFrame( )
# Common Clauses
released1Y = (lib.track_released>(datetime.now()-timedelta(days=365)))
released4M = (lib.track_released>(datetime.now()-timedelta(days=120)))
released3M = (lib.track_released>(datetime.now()-timedelta(days=90)))
added2W = (lib.added_at>(datetime.now()-timedelta(days=13)).isoformat())
added3M = (lib.added_at>(datetime.now()-timedelta(days=90)).isoformat())
added4M = (lib.added_at>(datetime.now()-timedelta(days=120)).isoformat())

# Debug playlists - used to exclude
noise = spot.fetchPlaylistUris("Noise")
decSongs = spot.fetchPlaylistUris("December Songs")
holidaySongs = spot.fetchPlaylistUris("Holiday Songs")



## Holiday Playlists
Maintain special "Holiday" playlists. These are comprised of holiday music and music associated with the holidays.


In [ ]:

allDecember = lib[(lib.original_uri.isin(decSongs)) | (lib.original_uri.isin(holidaySongs)) ]
holiday = lib[lib.original_uri.isin(holidaySongs)]

spot.updatePlaylist("December",allDecember)
spot.updatePlaylist("December Party", allDecember[ (allDecember.energy>0.55)  ])
spot.updatePlaylist("Holly Jolly", allDecember[ (allDecember.energy>0.25)  ])
spot.updatePlaylist("December Chill", allDecember[ (allDecember.energy<=0.25)  ])
spot.updatePlaylist("Silent Night", allDecember[ (allDecember.danceability<0.4) & (allDecember.energy<0.4) & (allDecember.instrumentalness>0.15) ])
spot.updatePlaylist("December Work", allDecember[ (allDecember.instrumentalness>0.15) & (allDecember.energy>0.2) ])
spot.updatePlaylist("December Instrumental", allDecember[ allDecember.instrumentalness>0.15 ])
spot.updatePlaylist("December Nonstrumental", allDecember[ allDecember.instrumentalness<=0.15 ])
spot.updatePlaylist("December 3M",holiday[ holiday.added_at>(datetime.now()-timedelta(days=90)).isoformat()],excludingPlaylists=[noise])
spot.updatePlaylist("December This Year",holiday[ holiday.track_released>(datetime.now()-timedelta(days=335))])
spot.updatePlaylist("December New",holiday[ holiday.track_released>(datetime.now()-timedelta(days=3.5*365))])
spot.updatePlaylist("December Classics",allDecember[ allDecember.track_released<(datetime(1991,1,1))])

In [ ]:
newPlaylist = lib[
    (lib.duration_ms<1000*60*7) & 
    (lib.energy>0.935) &
    (lib.track_released>datetime(1990,1,1))
].sort_values(["added_at"])
# Exclude some artists from this playlist
newPlaylist = newPlaylist[~newPlaylist.artist.isin(["Radiohead","Rodrigo y Gabriela","2CELLOS","Faithless","Doves"])]
spot.updatePlaylist("Auto Run Fast",newPlaylist,excludingPlaylists=[holidaySongs])
newRunFast = lib[
    released1Y &
     (lib.energy>0.80) 
    
].sort_values(["added_at"])
spot.updatePlaylist("New Run Fast",newRunFast,excludingPlaylists=[holidaySongs]).head(10)
lib[lib.artist.isin(["The Chemical Brothers"])][["track_album_name","track_released","track_name"]]

## Unpopular or Hispter?
This playlists tracks tracks with low popularity. This is useful for checking if you added the most commonly played track (e.g. a version before it was remastered).

In [ ]:
spot.updatePlaylist("Are You Sure",lib[(lib.track_popularity<2) & (lib.track_is_playable==True)],excludingPlaylists=[holidaySongs,noise,"New Released"])
spot.updatePlaylist("Unplayable",lib[(lib.track_is_playable==False)],excludingPlaylists=[])
comp = lib[~lib.track_album_name.isin(["Love","Ricochet (Live)"])]
spot.updatePlaylist("Compilations",comp[(comp.track_album_album_type=="compilation")],excludingPlaylists=[holidaySongs])


## Decades of Music
These playlists bucket the music in your library into decades using the release date. 

In [ ]:
spot.updatePlaylist("Auto 1960s",lib[lib.track_released<datetime(1970,1,1)],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Auto 1970s",lib[(lib.track_released>datetime(1970,1,1)) & (lib.track_released<datetime(1980,1,1))],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Auto 1980s",lib[(lib.track_released>datetime(1980,1,1)) & (lib.track_released<datetime(1990,1,1))],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Auto 1990s",lib[(lib.track_released>datetime(1990,1,1)) & (lib.track_released<datetime(2000,1,1))],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Auto 2000s",lib[(lib.track_released>datetime(2000,1,1)) & (lib.track_released<datetime(2010,1,1))],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Auto 2010s",lib[(lib.track_released>datetime(2010,1,1)) & (lib.track_released<datetime(2020,1,1))],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Auto 2020",lib[lib.track_released>=datetime(2020,1,1)],excludingPlaylists=[holidaySongs])


## Recently Added
The music you added in the last couple of months.

In [ ]:
spot.updatePlaylist("Good Year",lib[released1Y],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Auto 2W",lib[added2W],excludingPlaylists=[holidaySongs,noise])
spot.updatePlaylist("Auto 3M",lib[added3M],excludingPlaylists=[holidaySongs,noise,"Auto 2W"])
spot.updatePlaylist("New Released",lib[released3M],excludingPlaylists=[holidaySongs,noise])
spot.updatePlaylist("New Mix Takes",lib[released1Y & (lib.energy>0.5)],excludingPlaylists=[holidaySongs])

## Mood Music
Music for every occasion. Low energy dinner parties, going to sleep or concentrating at work.

In [ ]:
spot.updatePlaylist("Dinner Party",lib[(lib.acousticness>0.9) & (lib.energy<0.6)],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Night Nights",lib[(lib.acousticness>0.9) & (lib.loudness<-15) & (lib.speechiness<0.3) & (lib.energy<0.2) & (lib.danceability<0.3)],excludingPlaylists=[holidaySongs])

spot.updatePlaylist("Danceability",lib[ lib.danceability>0.90 ],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Energy",lib[ lib.energy>0.96 ],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Instrumentalness",lib[ lib.instrumentalness>0.96 ],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Low-Speechiness",lib[ lib.speechiness<0.025 ],excludingPlaylists=[holidaySongs])


workInstrumental = (lib.instrumentalness>0.65)

workDance = (lib.energy>0.7)
quietInstrumental = workInstrumental & (lib.energy<0.3)
quietDance = (lib.danceability<0.5)
quietEnergy = (lib.loudness<-15)
loudEnergy = (lib.loudness >= -15)
workFast = (lib.energy>0.8) & workInstrumental 

spot.updatePlaylist("To Work To",lib[workInstrumental & workDance ],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("To Work To 4M",lib[workInstrumental  & added4M],excludingPlaylists=[holidaySongs,noise])
spot.updatePlaylist("New Released To Work To",lib[workInstrumental & workDance & released1Y],excludingPlaylists=[holidaySongs,noise])

spot.updatePlaylist("Quiet Work",lib[quietInstrumental & quietDance & quietEnergy ],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Loud Work",lib[workInstrumental & workDance & loudEnergy],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Fast Work",lib[workFast],excludingPlaylists=[holidaySongs])


## More Analysis
It's easy to add the same track twice, from different albums. This can cause duplicates in playlists. We can use Pandas `groupby` to scan for that

In [ ]:
# Duplicates in the library
spot.updatePlaylist("Tangerine Dream",lib[ lib.artist.isin(["Tangerine Dream"])],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Duran Duran",lib[ lib.artist.isin(["Duran Duran","Arcadia"])],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Radiohead Plus",lib[ lib.artist.isin(["Radiohead","Thom Yorke","Jonny Greenwood","Modeselektor"]) | lib.track_album_name.str.contains("Radiohead") | lib.track_name.isin(['Faust Arp','Fake Plastic Trees','Idioteque','Exit Music','Everything in Its Right Place','True Love Waits','Exit Music (For A Film)','Weird Fishes']) ],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Beatles Plus",lib[ lib.artist.isin(["The Beatles","Paul McCartney","John Lennon","Ringo Starr","George Harrison","Wings","George Martin"]) | lib.track_album_name.str.contains("Across The Universe") | lib.track_album_name.str.contains("Instant Karma") | lib.track_album_name.str.contains("Abbey Road") | lib.track_album_name.str.contains("Beatles") | lib.track_name.isin(['Hard Day','Got to Get You Into My Life','Dear Prudence','Strawberry Fields',"Maybe I'm Amazed","Golden Slumbers","Here Comes the Sun","A Day in the Life"]) ],excludingPlaylists=[holidaySongs])
spot.updatePlaylist("Movies",lib[ lib.artist.isin(["Hans Zimmer","Trent Reznor","Trent Reznor and Atticus Ross","John Williams","Howard Shore","Ennio Morricone","James Horner","Steven Price","Carter Burwell","John Barry","Vangelis","Thomas Newman","Philip Glass","Craig Armstrong","Jóhann Jóhannsson","Michael Giacchino"]) ],excludingPlaylists=[holidaySongs])

sm = lib[["track_name","artist","track_duration_ms","track_id","original_id","track_uri","original_uri","track_album_name","track_album_album_type"]]
ff = sm.groupby(["track_name","artist","track_duration_ms"])["track_id"]

In [ ]:
lib.columns.values
#lib[["artistGenres"]].head(200)

# lib[lib.track_album_name.isin(["Circles"])][["artist","track_name","speechiness","instrumentalness","danceability","energy","track_uri","track_id","original_id"]]